In [1]:
import pyvisa as pv
import numpy as np
from matplotlib import pyplot as plt
# from pymeasure.instruments import keithley as kt
import time

drain_address = "USB0::0x05E6::0x2450::04096333::INSTR"
gate_address = "USB0::0x05E6::0x2450::04096331::INSTR"

def set_smu_ready(address):
    rm = pv.ResourceManager()
    smu = rm.open_resource(address)
    # smu = kt.Keithley2450(address)
    smu.write('reset()')
    smu.write('smu.source.func = smu.FUNC_DC_VOLTAGE')
    smu.write('smu.source.autorange=smu.ON')
    return smu
def write_list(smu, input_list, delay=0.0001):
    smu.write("smu.source.configlist.create(\"CustomSweep\")")
    for volt in input_list:
        smu.write('smu.source.level = ' + str(volt))
        smu.write('smu.source.configlist.store(\"CustomSweep\")')
    smu.write('smu.source.sweeplist(\"CustomSweep\",1,' + str(delay) +')')
    # smu.write('trigger.model.setblock(3,trigger.BLOCK_DELAY_CONSTANT, 0)')
    # smu.write('trigger.model.setblock(10,trigger.BLOCK_DELAY_CONSTANT, 0)')
    # return smu.query('print(trigger.model.getblocklist())')

def write_volt(smu,volt=0):
    smu.write(f'smu.source.level = {volt}')
    smu.write('smu.measure.read()')

def ramp(smu, current_volt=0,target_volt=1, steps=10, ramp_time=1):
    # if smu.query('print(smu.source.output)')!= 'smu.ON\n':
    #     print('Error: SMU output not turned on.')
    #     smu.write('smu.source.output = smu.ON')
    # current_volt = smu.query_ascii_values('print(smu.source.level)')[0]
    volt_list = np.linspace(current_volt,target_volt,num=steps)
    for volt in volt_list:
        write_volt(smu=smu, volt=volt)
        time.sleep(ramp_time/steps)
    smu.write('waitcomplete()')
    # if target_volt ==0:
    #     print('SMU is at 0 outout. Output has been turned off.')


In [3]:
# gate = set_smu_ready(gate_address,measure_current=False)
drain = set_smu_ready(drain_address)

input_list = np.linspace(-0.1,0.1,num=10)
drain.write('smu.measure.func = smu.FUNC_DC_VOLTAGE')
drain.write('smu.source.level = 0')
drain.write('smu.source.output = smu.ON')
ramp(smu=drain, current_volt=0, target_volt=-1,ramp_time=0.5)
status = write_list(drain, input_list)
drain.write('smu.measure.func = smu.FUNC_DC_CURRENT')
drain.write('trigger.model.initiate()')
drain.write('waitcomplete()')
drain.write('smu.measure.func = smu.FUNC_DC_VOLTAGE')
ramp(smu=drain, current_volt=1, target_volt=0,ramp_time=0.5)
drain.write('smu.source.output = smu.OFF')
# print(drain.query('smu.source.output'))
# data = drain.query_ascii_values('printbuffer(1, defbuffer1.n, defbuffer1.sourcevalues, defbuffer1.readings)')
# ramp(smu=drain, target_volt=0,ramp_time=0.2)
# drain.write('smu.source.output = smu.OFF')

29

In [ ]:
data = drain.query_ascii_values('printbuffer(1, defbuffer1.n, defbuffer1.sourcevalues, defbuffer1.readings)')

VisaIOError: VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.

In [ ]:
# drain.query('print(smu.source.output)')

In [ ]:
np.array([np.array(data)[::2],np.array(data)[1::2]]).T.shape

(160, 2)

In [5]:
print(drain.query('print(trigger.model.getblocklist())'))

 1) BUFFER_CLEAR          BUFFER: defbuffer1
 2) CONFIG_RECALL         CONFIG_LIST: CustomSweep  INDEX: 1
 3) SOURCE_OUTPUT         OUTPUT: ON
 4) BRANCH_ALWAYS         BRANCH_BLOCK: 6 
 5) CONFIG_NEXT           CONFIG_LIST: CustomSweep
 6) BRANCH_ON_EVENT       EVENT: EVENT_SOURCE_LIMIT  BRANCH_BLOCK: 12
 7) DELAY_CONSTANT        DELAY: 0.000100000
 8) MEASURE_DIGITIZE      BUFFER: defbuffer1 INITIAL MODE: MEAS INITIAL COUNT: 1
 9) BRANCH_COUNTER        VALUE: 10  BRANCH_BLOCK: 5 
10) SOURCE_OUTPUT         OUTPUT: OFF
11) BRANCH_ALWAYS         BRANCH_BLOCK: 0 
12) SOURCE_OUTPUT         OUTPUT: OFF
13) LOG_EVENT             EVENT: # 2733  MESSAGE: "source limit exceeded"

* This trigger model may not be set up for low latency. *


